In [7]:
!pip install mediapipe
!pip install chumpy
!pip install protobuf==3.20.*
!pip install opencv-python

In [7]:
# import related package and initialize Mediapipe hand

import cv2
import numpy as np
import chumpy as ch
import mediapipe as mp
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import os
import torch
%matplotlib inline
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

HANDS_IDX = [2, 3]

In [4]:
img_path = 'C:/Users/Zaman/Documents/GitHub Repos/project-jarvis/landmarks/s4_t2/raw_images'
bbox_path = 'C:/Users/Zaman/Documents/GitHub Repos/project-jarvis/landmarks/s4_t2/bounding_box'

img_list = os.listdir(img_path)
bbox_list = os.listdir(bbox_path)

count = 10
img_list = img_list[:count]
bbox_list = bbox_list[:count]

for c in range(count):
    img_list[c] = img_path + '/' + img_list[c]
    bbox_list[c] = bbox_path + '/' + bbox_list[c]

In [5]:
def get_bbox_rl(bbox_filepath, image_width, image_height):
    file = open(bbox_filepath, 'r')
    Lines = file.readlines()

    bbox_r = []
    bbox_l = []
    
    # Strips the newline character
    for line in Lines:
        l = line.strip()
        l = l.split()
        if l[0] == str(HANDS_IDX[0]):
            for i in range(4):
                bbox_l.append(float(l[i+1]))
        elif l[0] == str(HANDS_IDX[1]):
            for i in range(4):
                bbox_r.append(float(l[i+1]))

    # [x_center, y_center, width, height]
    # TO
    # [x1, x2, y1, y2]

    x1 = int((bbox_r[0] - bbox_r[2]/2) * image_width)
    x2 = int((bbox_r[0] + bbox_r[2]/2) * image_width)
    y1 = int((bbox_r[1] - bbox_r[3]/2) * image_height)
    y2 = int((bbox_r[1] + bbox_r[3]/2) * image_height)

    bbox_r = [x1, x2, y1, y2]

    x1 = int((bbox_l[0] - bbox_l[2]/2) * image_width)
    x2 = int((bbox_l[0] + bbox_l[2]/2) * image_width)
    y1 = int((bbox_l[1] - bbox_l[3]/2) * image_height)
    y2 = int((bbox_l[1] + bbox_l[3]/2) * image_height)

    bbox_l = [x1, x2, y1, y2]

    return bbox_r, bbox_l

In [8]:
IMAGE_FILES = img_list
ANNOTATED_IMAGES = []

with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.5) as hands:

        for idx, file in enumerate(IMAGE_FILES):
            # Read an image, flip it around y-axis for correct handedness output (see
            # above).
            image = cv2.imread(file)
            
            bbox_r, bbox_l = get_bbox_rl(bbox_list[idx], image.shape[1], image.shape[0])
            image = image[bbox_l[2]:bbox_l[3], bbox_l[0]:bbox_l[1]]
            annotated_image = image.copy()

            # Convert the BGR image to RGB before processing.
            results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            # Print handedness and draw hand landmarks on the image.
            # print('Handedness:', results.multi_handedness)
            if not results.multi_hand_landmarks:
                ANNOTATED_IMAGES.append(annotated_image)
                continue
            image_height, image_width, _ = image.shape
            for hand_landmarks in results.multi_hand_landmarks:
                # print('hand_landmarks:', hand_landmarks)
                # print(
                #     f'Index finger tip coordinates: (',
                #     f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
                #     f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'
                # )
                mp_drawing.draw_landmarks(
                    annotated_image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())

            ANNOTATED_IMAGES.append(annotated_image)

print(len(ANNOTATED_IMAGES))


7


In [77]:
# Output format
x = results.multi_handedness[0]
print(x.classification[0].label)

Right


In [78]:
right_landmarks_idx = None
left_landmarks_idx = None

count = 0
while (count < len(results.multi_handedness)):
    l = results.multi_handedness[count].classification[0].label
    if l == "Right":
        right_landmarks_idx = results.multi_handedness[count].classification[0].index
    elif l == "Left":
        left_landmarks_idx = results.multi_handedness[count].classification[0].index
    count += 1

print(right_landmarks_idx, left_landmarks_idx)

1 None


In [79]:
class Landmark:
    def __init__(self, image_width, image_height, x, y, z):
        self.x = round(x * image_width)
        self.y = round(y * image_height)
        self.z = round(z)

    def output_dict(self):
        d = {
            "x" : self.x,
            "y" : self.y,
            # "z" : self.z 
            }
        return d

right_landmarks = []
left_landmarks = []
idx = 0

for hand_landmarks in results.multi_hand_landmarks:
    for l in hand_landmarks.landmark:
        l_obj = Landmark(image_width, image_height, l.x, l.y, l.z)
        if right_landmarks_idx == idx:
            right_landmarks.append(l_obj.output_dict())
        else:
            left_landmarks.append(l_obj.output_dict())
    idx += 1

print("Right hand landmarks: ")  
print(right_landmarks)

print("Left hand landmarks: ")
print(left_landmarks)

Right hand landmarks: 
[]
Left hand landmarks: 
[{'x': 93, 'y': 122}, {'x': 119, 'y': 95}, {'x': 126, 'y': 62}, {'x': 129, 'y': 40}, {'x': 129, 'y': 21}, {'x': 77, 'y': 46}, {'x': 68, 'y': 20}, {'x': 72, 'y': 7}, {'x': 80, 'y': 2}, {'x': 64, 'y': 56}, {'x': 59, 'y': 32}, {'x': 67, 'y': 21}, {'x': 79, 'y': 18}, {'x': 59, 'y': 67}, {'x': 56, 'y': 45}, {'x': 63, 'y': 35}, {'x': 71, 'y': 31}, {'x': 60, 'y': 75}, {'x': 56, 'y': 56}, {'x': 61, 'y': 47}, {'x': 67, 'y': 44}]
